# Pair Programming Limpieza II

In [126]:
import pandas as pd
pd.options.display.max_columns = None
import re
import sidetable
import numpy as np

In [127]:
df = pd.read_csv('../datos/datos_limpiezaI.csv', index_col= 0)
df.head(1)

,year,country,age,species_,date,mes,fatal,sexo
0,2018,usa,57,White shark,25-Jun-2018,Jun,n,F


In [128]:
df.rename(columns = {"species_": "species","mes": "month","sexo": "sex" }, inplace = True)
df.columns
#cambiamos los nombres de todas las columnas a ingles

Index(['year', 'country', 'age', 'species', 'date', 'month', 'fatal', 'sex'], dtype='object')

1. Columna de species: si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. 

    - Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (*White*), el tiburón tigre (*Tiger*), el tiburón gris (*Grey*), el tiburón limón (*Lemon*) y el tiburón toro (*Bull*). 

    - El resto de las especies las incluiremos en un único grupo que podremos llamar *"Unspecified"*. Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.

    Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.

- Tendremos que crearnos una función que aplicaremos sobre nuestra columna species para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [129]:
def categorizar (col):

    patron_blanco = re.findall(r".*[Ww](hite|HITE).*", str(col))
    patron_tigre = re.findall(r".*[Tt](iger|IGER).*", str(col))
    patron_gris = re.findall(r".*[Gg](rey|REY).*", str(col))
    patron_limon = re.findall(r".*[Ll](emon|EMON).*", str(col))
    patron_toro = re.findall(r".*[Bb](ull|ULL).*", str(col))

    if patron_blanco:
        return "white_shark"
    elif patron_tigre:
        return "tiger_shark"
    elif patron_gris:
        return "grey_shark"
    elif patron_limon:
        return "lemon_shark"
    elif patron_toro:
        return "bull_shark"
    else:
        return str("unspecified")



In [130]:
df["main_species"] = df["species"].apply(categorizar)

In [131]:
df.head()
#averiguamos se haya creado la columna main_species.

,year,country,age,species,date,month,fatal,sex,main_species
0,2018,usa,57,White shark,25-Jun-2018,Jun,n,F,white_shark
1,2018,brazil,18,Tiger shark,03-Jun-2018,Jun,y,M,tiger_shark
2,2018,usa,15,"Bull shark, 6'",26-May-2018,May,n,M,bull_shark
3,2018,australia,32,Grey reef shark,24-May-2018,May,n,M,grey_shark
4,2018,england,21,Invalid incident,13-May-2018,May,n,M,unspecified


In [132]:
df.stb.freq(["main_species"],style=True)
#averiguamos el % de cadas especies

,main_species,count,percent,cumulative_count,cumulative_percent
0,unspecified,978,58.70%,978,58.70%
1,white_shark,386,23.17%,"1,364",81.87%
2,tiger_shark,157,9.42%,"1,521",91.30%
3,bull_shark,104,6.24%,"1,625",97.54%
4,grey_shark,30,1.80%,"1,655",99.34%
5,lemon_shark,11,0.66%,"1,666",100.00%


2. Columna de *age*: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:
    - Edad en formato string
    - Edades separadas por &, or, to, >
    - Edades con ?

Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:
- Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.
- Puede que os salga un error similar a este:
    - TypeError: expected string or bytes-like object
    - Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:
    - df['nombre_columna'] = df['nombre_columna'].astype(str)
- Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método explode, aqui.
- Por último cambiad el tipo de la columna de string a integer.

In [133]:
df['age'] = df['age'].astype(str)

In [134]:
#def limpia_edad(edad):
    #patron_r = re.findall(r"\d+", str(edad))
    #patron_r = "\d+"
    #try:
     #   return re.findall(patron_r, str(edad))
    #except:
     #   np.nan    


In [135]:
def limpia_edad(edad):
    #patron_r = re.findall(r"\d+", str(edad))
    patron_r = "\d+"
    try:
        return re.findall(patron_r, str(edad))[0]    
    except:
        np.nan    

In [136]:
df["age_numbers"]=(df["age"].apply(limpia_edad))

In [137]:
df['age_numbers'] = df['age_numbers'].astype("int", errors = 'ignore')

In [146]:
#averiguamos los unicos
df['age_numbers'].unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '11', '13',
       '34', '50', '46', '48', '17', '28', '65', '73', '58', '36', '51',
       '61', '59', '42', '6', '27', '23', '29', '39', '24', '12', '26',
       '71', '43', '9', '44', '14', '62', '52', '38', '68', '16', '47',
       '63', '70', '41', '53', '7', '66', '45', '74', '64', '8', '56',
       '49', None, '77', '84', '5', '67'], dtype=object)

3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.

In [159]:
df.to_csv("../datos/datos_limpiezaII.csv", index = False)